$B \rightarrow K^* \ell \ell$  machine learning experiment

Setup

In [ ]:
from torch.nn import MSELoss, CrossEntropyLoss
import matplotlib.pyplot as plt

from helpers.datasets.make_and_save.aggregated_signal import Aggregated_Signal_Dataframe_Handler
from helpers.datasets.constants import Names_of_Levels, Names_of_q_Squared_Vetos, Raw_Signal_Trial_Ranges, Numbers_of_Events_per_Set, Names_of_Splits, Names_of_Labels
from helpers.experiment.experiment import CNN_Group, Deep_Sets_Group, Event_by_Event_Group
from helpers.experiment.results_table import Results_Table
from helpers.experiment.constants import Paths_to_Directories
from helpers.models.hardware_util import select_device
from helpers.experiment.experiment import evaluate_model
from helpers.datasets.settings.settings import Binned_Sets_Dataset_Settings
from helpers.datasets.datasets import Unbinned_Sets_Dataset, Binned_Sets_Dataset
from helpers.datasets.make_and_save.preprocessing import apply_q_squared_veto

results_table = Results_Table()
device = select_device()

Remake aggregated signal dataframe files

In [ ]:
for  level in (Names_of_Levels().generator, Names_of_Levels().detector):
    for trial_range in Raw_Signal_Trial_Ranges().tuple_:
        
        Aggregated_Signal_Dataframe_Handler(
            path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
            level=level,
            trial_range=trial_range
        ).make_and_save(Paths_to_Directories().path_to_raw_signal_dir)

Deep Sets

In [ ]:
deep_sets_group = Deep_Sets_Group(
    num_sets_per_label={6_000 : 583, 24_000 : 145, 70_000 : 50},
    num_sets_per_label_sensitivity=2_000,
    q_squared_veto=Names_of_q_Squared_Vetos().resonances,
    std_scale=True,
    shuffle=True,
    uniform_label_counts=True,
    loss_fn=MSELoss(),
    learning_rate=3e-4,
    learning_rate_scheduler_reduction_factor=0.97,
    size_of_training_batch={6_000 : 373, 24_000 : 93, 70_000 : 32},
    size_of_evaluation_batch={6_000 : 373, 24_000 : 93, 70_000 : 32},
    number_of_epochs=100,
    number_of_epochs_between_checkpoints=1,
    results_table=results_table,
    device=device,
    bkg_fraction=0.2,
    bkg_charge_fraction=0.5
)

deep_sets_group.train_subset(
    levels=(Names_of_Levels().detector, Names_of_Levels().generator),
    nums_events_per_set=(6_000,),
    remake_datasets=True
)

deep_sets_group.evaluate_subset(
    levels=(Names_of_Levels().detector, Names_of_Levels().generator), 
    nums_events_per_set=(6_000,), 
    remake_datasets=True,
    epoch="final"
)

In [ ]:
deep_sets_group.train_subset(
    levels=Names_of_Levels().tuple_,
    nums_events_per_set=(24_000, 70_000),
    remake_datasets=True
)

deep_sets_group.evaluate_subset(
    levels=Names_of_Levels().tuple_, 
    nums_events_per_set=(24_000, 70_000), 
    remake_datasets=True,
    epoch="final"
)

CNN

In [ ]:
cnn_group = CNN_Group(
    num_sets_per_label={6_000 : 583, 24_000 : 145, 70_000 : 50},
    num_sets_per_label_sensitivity=2_000,
    num_bins_per_dimension=10,
    q_squared_veto=Names_of_q_Squared_Vetos().resonances,
    std_scale=True,
    shuffle=True,
    uniform_label_counts=True,
    loss_fn=MSELoss(),
    learning_rate=3e-4,
    learning_rate_scheduler_reduction_factor=0.95,
    size_of_training_batch={6_000 : 373, 24_000 : 93, 70_000 : 32},
    size_of_evaluation_batch={6_000 : 373, 24_000 : 93, 70_000 : 32},
    number_of_epochs=50,
    number_of_epochs_between_checkpoints=1,
    results_table=results_table,
    device=device,
    bkg_fraction=0.2,
    bkg_charge_fraction=0.5
)

cnn_group.train_subset(levels=[Names_of_Levels().detector,], nums_events_per_set=[6_000,], remake_datasets=True)
cnn_group.evaluate_subset(levels=[Names_of_Levels().detector,], nums_events_per_set=[6_000,], remake_datasets=True)
cnn_group.plot_image_examples_subset(levels=[Names_of_Levels().detector,], nums_events_per_set=[6_000,], remake_datasets=False)

# cnn_group.train_all(remake_datasets=True)
# cnn_group.evaluate_all(remake_datasets=True)
# cnn_group.plot_image_examples_all(remake_datasets=False)

In [ ]:
cnn_group.train_subset(levels=[Names_of_Levels().detector_and_background,], nums_events_per_set=[6_000,], remake_datasets=True)
cnn_group.evaluate_subset(levels=[Names_of_Levels().detector_and_background,], nums_events_per_set=[6_000,], remake_datasets=True)
cnn_group.plot_image_examples_subset(levels=[Names_of_Levels().detector_and_background,], nums_events_per_set=[6_000,], remake_datasets=False)

In [ ]:
cnn_group.train_subset(levels=Names_of_Levels().tuple_, nums_events_per_set=[24_000, 70_000], remake_datasets=True)
cnn_group.evaluate_subset(levels=Names_of_Levels().tuple_, nums_events_per_set=[24_000, 70_000], remake_datasets=True)
cnn_group.plot_image_examples_subset(levels=Names_of_Levels().tuple_, nums_events_per_set=[24_000, 70_000], remake_datasets=False)

Event-by-event

In [ ]:
event_by_event_group = Event_by_Event_Group(
    num_evaluation_sets_per_label=50,
    num_evaluation_sets_per_label_sensitivity=2_000,
    q_squared_veto=Names_of_q_Squared_Vetos().resonances,
    std_scale=True,
    shuffle=True,
    uniform_label_counts=True,
    loss_fn=CrossEntropyLoss(),
    learning_rate=3e-3,
    learning_rate_scheduler_reduction_factor=0.95,
    size_of_training_batch=10_000,
    size_of_evaluation_batch=10_000,
    number_of_epochs=300,
    number_of_epochs_between_checkpoints=2,
    results_table=results_table,
    device=device
)

# event_by_event_group.train_subset(levels=[Names_of_Levels.detector,], remake_datasets=True)

event_by_event_group.evaluate_subset(
    levels=[Names_of_Levels().detector,], 
    nums_events_per_set=[6_000, 24_000, 70_000], 
    remake_datasets=True,
    epoch="final"
)

In [ ]:
import pandas
import matplotlib.pyplot as plt

In [ ]:
charge_train = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")
mix_train = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")
all_train = pandas.concat([charge_train, mix_train])

charge_eval = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")
mix_eval = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")
all_eval = pandas.concat([charge_eval, mix_eval])

charge_eval = apply_q_squared_veto(charge_eval, Names_of_q_Squared_Vetos().resonances)
charge_train = apply_q_squared_veto(charge_train, Names_of_q_Squared_Vetos().resonances)

In [ ]:
charge_train

In [ ]:
mix_train

In [ ]:
charge_eval.count()

In [ ]:
charge_eval.std()

In [ ]:
cut(all_train["q_squared"])

In [ ]:
cut(all_eval["q_squared"])

In [ ]:
for  level in (Names_of_Levels().generator, Names_of_Levels().detector):
    for trial_range in Raw_Signal_Trial_Ranges().tuple_:
        print(level, trial_range)
        Aggregated_Signal_Dataframe_Handler(
            path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
            level=level,
            trial_range=trial_range
        ).get_dataframe()[Names_of_Labels().unbinned].hist(bins=20)

In [ ]:
dataframe = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().detector,
    trial_range=Raw_Signal_Trial_Ranges().train
).get_dataframe()

dataframe = apply_q_squared_veto(dataframe, Names_of_q_Squared_Vetos().resonances)

label_counts = dataframe[Names_of_Labels().unbinned].value_counts()

label_counts.values.min()

# plt.scatter(label_counts.index, label_counts.values)
# plt.ylim(0, 180_000)

In [ ]:
dataframe

In [ ]:
def reduce_num_per_label_to_lowest(dataframe):

    num_lowest = dataframe[Names_of_Labels().unbinned].value_counts().values.max()
    get_subset = lambda x : x.iloc[:num_lowest]
    dataframe = (
        dataframe.groupby(Names_of_Labels().unbinned, group_keys=False)[dataframe.columns]
        .apply(get_subset)
    )
    return dataframe

reduce_num_per_label_to_lowest(dataframe)

In [ ]:
tuple({3 : 1, 4 : 2}.keys())
